In [1]:
import numpy as np 
import re
from scipy.interpolate import CubicSpline
from astropy.table import Table
import astropy.units as u
from astropy.cosmology import FlatLambdaCDM
from astropy.constants import c
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt

In [2]:
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

In [3]:
#Read all the files from the SED modeling. 
folder = "../../SED_Modeling"

#Band information
bands = Table.read("{}/bandmag.dat".format(folder), format='ascii')
for i, hn in enumerate(['bname', 'bcal', 'jyzero']):
    bands.rename_column("col{}".format(i+1), hn)
bands['bname'][-2] = "g_misc"
bands['bname'][-1] = "r_misc"

#SED model parameters
d20 = Table.read("{}/double.20".format(folder), format='ascii')
for i, hn in enumerate(['i', 'redshift', 'chi2', 'vec1', 'vec2', 'vec3', 'vec4', 'vec5']):
    d20.rename_column("col{}".format(i+1), hn)
d22 = Table.read("{}/double.22".format(folder), format='ascii')
for i, hn in enumerate(['i', 'ebv1', 'ebv2', 'igm', 'wid']):
    d22.rename_column("col{}".format(i+1), hn)

#The Pran probs
pran = dict()
cat = open("{}/f_stat/comp.sort".format(folder))
for line in cat:
    x = line.split()
    pran[x[0]] = float(x[-1])

In [4]:
#Get the errors. 
mc_d20 = Table.read("{}/MC/double.20".format(folder), format='ascii')
for i, hn in enumerate(['i', 'redshift', 'chi2', 'vec1', 'vec2', 'vec3', 'vec4', 'vec5']):
    mc_d20.rename_column("col{}".format(i+1), hn)

mc_d22 = Table.read("{}/MC/double.22".format(folder), format='ascii')
for i, hn in enumerate(['i', 'ebv1', 'ebv2', 'igm']):
    mc_d22.rename_column("col{}".format(i+1), hn)

nrep = int(len(mc_d20)/len(d20))
probs = np.array([68.3, 95.4, 99.7])
intervals = np.concatenate([50-probs/2., 50+probs/2.])
for col in ['vec1', 'vec5']:
    for j in range(3):
        d20[col+'_l{}'.format(j+1)] = 0.
        d20[col+'_u{}'.format(j+1)] = 0.
    for i in range(len(d20)):
        aux = np.percentile(mc_d20[col][i*nrep:(i+1)*nrep], intervals)
        for j in range(3):
            d20[col+'_l{}'.format(j+1)][i] = d20[col][i] - aux[j]
            d20[col+'_u{}'.format(j+1)][i] = aux[j+3] - d20[col][i]

for col in ['ebv1', 'ebv2']:
    for j in range(3):
        d22[col+'_l{}'.format(j+1)] = 0.
        d22[col+'_u{}'.format(j+1)] = 0.
    for i in range(len(d22)):
        aux = np.percentile(mc_d22[col][i*nrep:(i+1)*nrep], intervals)
        for j in range(3):
            d22[col+'_l{}'.format(j+1)][i] = d22[col][i] - aux[j]
            d22[col+'_u{}'.format(j+1)][i] = aux[j+3] - d22[col][i]


# for col in ['evec1_u', 'evec1_l', 'evec5_u', 'evec5_l']:
#     d20[col] = 0.0
# for col in ['ebv1_u', 'ebv1_l', 'ebv2_u', 'ebv2_l']:
#     d22[col] = 0.0

# nrep = int(len(mc_d20)/len(d20))
# print(nrep)
# for i in range(len(d20)):
#     d20['evec1_u'][i] = np.percentile(mc_d20['vec1'][i*nrep:(i+1)*nrep], 50.+68.3/2)
#     d20['evec1_l'][i] = np.percentile(mc_d20['vec1'][i*nrep:(i+1)*nrep], 50.-68.3/2)
#     d20['evec5_u'][i] = np.percentile(mc_d20['vec5'][i*nrep:(i+1)*nrep], 50.+68.3/2)
#     d20['evec5_l'][i] = np.percentile(mc_d20['vec5'][i*nrep:(i+1)*nrep], 50.-68.3/2)
#     d22['ebv1_u'][i]  = np.percentile(mc_d22['ebv1'][i*nrep:(i+1)*nrep], 50.+68.3/2)
#     d22['ebv1_l'][i]  = np.percentile(mc_d22['ebv1'][i*nrep:(i+1)*nrep], 50.-68.3/2)
#     d22['ebv2_u'][i]  = np.percentile(mc_d22['ebv2'][i*nrep:(i+1)*nrep], 50.+68.3/2)
#     d22['ebv2_l'][i]  = np.percentile(mc_d22['ebv2'][i*nrep:(i+1)*nrep], 50.-68.3/2)

# d20['evec1_u'] = d20['evec1_u'] - d20['vec1']
# d20['evec1_l'] = d20['vec1'] - d20['evec1_l']
# d20['evec5_u'] = d20['evec5_u'] - d20['vec5']
# d20['evec5_l'] = d20['vec5'] - d20['evec5_l']
# d22['ebv1_u']  = d22['ebv1_u'] - d22['ebv1']
# d22['ebv1_l']  = d22['ebv1'] - d22['ebv1_l']
# d22['ebv2_u']  = d22['ebv2_u'] - d22['ebv2']
# d22['ebv2_l']  = d22['ebv2'] - d22['ebv2_l']


In [5]:
d22.show_in_notebook()

idx,i,ebv1,ebv2,igm,wid,ebv1_l1,ebv1_u1,ebv1_l2,ebv1_u2,ebv1_l3,ebv1_u3,ebv2_l1,ebv2_u1,ebv2_l2,ebv2_u2,ebv2_l3,ebv2_u3
0,1,5.011872,0.03162278,1.4,W0019-1046,0.14099985000000004,0.42288210149999994,0.5448948020000008,0.8577767009999997,1.0308000000000002,1.297701,0.03162278,0.04781004,0.03162278,0.06837722,0.03162278,0.06837722
1,2,4.323613,0.0,0.4451524,W0116-0505,0.8639302015000001,1.4717886435,1.3147961890000004,3.8736991530000022,1.8117269999999999,5.854741084999928,0.0,0.0,0.0,0.01584893,0.0,0.03162278
2,3,10.0,0.1,0.0,W0204-0506,2.056718,0.2073381350000023,3.6904269999999997,2.58925,4.27541458100001,5.848929999999999,0.08415107000000001,0.0,0.1,0.0,0.1,0.0
3,4,7.570073,0.0,0.3727581,W0220+0137,1.0164122620000002,0.8421383755000003,1.680391815000001,1.9909794260000018,2.5582009999999995,2.6813896949999743,0.0,0.0,0.0,0.01,0.0,0.01995262
4,5,3.162278,0.01584893,1.4,W0831+0140,0.12021254699999995,0.48095612550000055,0.6503920000000001,0.818794,0.6503920000000001,1.3459126004999997,0.01584893,0.015773850000000002,0.01584893,0.034269789999999994,0.01584893,0.06358389


In [6]:
#Read the photometry
d21_raw = np.loadtxt("{}/double.21".format(folder))
nchan = len(bands)
d21 = Table()
d21['wid'] = d22['wid']
for k, bname in enumerate(bands['bname']):
    d21[bname] = d21_raw[k::nchan, 1] * u.Jy
    d21[bname+" mod"] = d21_raw[k::nchan, 2] * u.Jy
    d21[bname+" err"] = d21_raw[k::nchan, 3] * u.Jy

In [7]:
dbase = Table.read("{}/dbase.fits".format(folder))
coords = dict()
for wid in pran.keys():
    k = np.where(dbase['Short Name']==wid)[0][0]
    coords[wid] = SkyCoord(ra=dbase['ra'][k]*u.deg, dec=dbase['dec'][k]*u.deg)

In [8]:
coords['W0116-0505'].to_string(style='hmsdms', sep=":")

'01:16:01.411584 -05:05:04.09308'

In [9]:
#We now want to get the 6um luminosities of the unreddened best-fit quasar templates. 
seds = np.loadtxt("{}/agn_spec.dat".format(folder), skiprows=1)
agn_sed = CubicSpline(seds[:,0], seds[:,2])

In [10]:
# #Calculate the 6um luminosities for each target. 
# z = d20['redshift']
# DL = cosmo.luminosity_distance(z)
# for j, vec in enumerate(['vec1', 'vec5']):
#     f_nu = d20[vec]*agn_sed(6.0) * u.Jy
#     L_nu = (4.*np.pi*DL**2)/(1+z) * f_nu
#     L6um = (c/(6*u.micron) * L_nu).to(u.erg/u.s)
#     d22['log L6um AGN{}'.format(j+1)] = np.log10(L6um.value)

In [11]:
d20.show_in_notebook()

idx,i,redshift,chi2,vec1,vec2,vec3,vec4,vec5,vec1_l1,vec1_u1,vec1_l2,vec1_u2,vec1_l3,vec1_u3,vec5_l1,vec5_u1,vec5_l2,vec5_u2,vec5_l3,vec5_u3
0,1,1.641,5.081521,0.002470636,7.007495e-06,0.0,3.758273e-06,2.793912e-05,0.00014495672600000004,0.00021030723899999977,0.00030678725800000004,0.0004207122460000005,0.0005168843965000004,0.0006125039029999816,7.859589619999998e-06,1.9905227835000012e-05,1.375092389e-05,3.686418382e-05,1.9288139319000003e-05,4.863015567499957e-05
1,2,3.173,24.02249,0.004411795,3.869816e-05,0.0,0.0,3.837841e-05,0.0007090544889999995,0.001287889304500002,0.001209945889,0.003569997600000001,0.0015891211240000001,0.0055030767924999745,4.498895544999997e-06,2.047541885000002e-06,1.2814494380000004e-05,7.466635490000005e-06,2.1734375955000034e-05,2.0353979559999968e-05
2,3,2.099307,17.83592,0.003593931,0.0,2.776747e-05,0.0,4.697634e-05,0.0007784559130000002,0.00034741796200000053,0.0011567671220000003,0.0012159665510000006,0.001494806261500001,0.002356434606499971,3.1753156165000005e-05,5.464587689999998e-06,4.1594108950000004e-05,1.4103446939999994e-05,4.697634e-05,2.2120534429999832e-05
3,4,3.122,6.049306,0.005571716,1.533589e-05,8.421436e-07,0.0,2.792184e-05,0.0007453685604999994,0.0007250514835000004,0.0013044490819999997,0.0016656779980000003,0.001921120264500001,0.002468571265499992,1.3148636299999978e-06,8.445419449999993e-07,2.9726564600000012e-06,3.075603220000016e-06,5.004312690000003e-06,7.594921859999814e-06
4,5,3.888,5.716598,0.005847326,1.819741e-05,0.0,1.039276e-05,3.070182e-05,0.0002729877520000002,0.0006337925315000005,0.0007394866340000002,0.0012136039870000006,0.001175488994,0.001888996610499987,8.245521705000001e-06,1.7145428275000004e-05,1.4354663520000003e-05,3.463095247e-05,1.8291137079999998e-05,5.875637567999972e-05


In [12]:
#Calculate the 6um luminosities for each target. 
z = d20['redshift']
DL = cosmo.luminosity_distance(z)
for j, vec in enumerate(['vec1', 'vec5']):
    f_nu = d20[vec]*agn_sed(6.0) * u.Jy
    L_nu = (4.*np.pi*DL**2)/(1+z) * f_nu
    L6um = (c/(6*u.micron) * L_nu).to(u.erg/u.s)
    d22['log L6um AGN{}'.format(j+1)] = np.log10(L6um.value)
    for k in range(3):
        d22['log L6um AGN{}_l{}'.format(j+1,k+1)] = np.log10(L6um.value) - np.log10(L6um.value * (d20[vec]-d20[vec+"_l{}".format(k+1)])/d20[vec])
        d22['log L6um AGN{}_u{}'.format(j+1,k+1)] = np.log10(L6um.value * (d20[vec]+d20[vec+"_u{}".format(k+1)])/d20[vec]) - np.log10(L6um.value)

/var/folders/p7/drxzchtj4yb641v79lt0tjyh0000gn/T/ipykernel_89098/4061056990.py:10: RuntimeWarning: divide by zero encountered in log10
  d22['log L6um AGN{}_l{}'.format(j+1,k+1)] = np.log10(L6um.value) - np.log10(L6um.value * (d20[vec]-d20[vec+"_l{}".format(k+1)])/d20[vec])


In [13]:
d22.show_in_notebook()

idx,i,ebv1,ebv2,igm,wid,ebv1_l1,ebv1_u1,ebv1_l2,ebv1_u2,ebv1_l3,ebv1_u3,ebv2_l1,ebv2_u1,ebv2_l2,ebv2_u2,ebv2_l3,ebv2_u3,log L6um AGN1,log L6um AGN1_l1,log L6um AGN1_u1,log L6um AGN1_l2,log L6um AGN1_u2,log L6um AGN1_l3,log L6um AGN1_u3,log L6um AGN2,log L6um AGN2_l1,log L6um AGN2_u1,log L6um AGN2_l2,log L6um AGN2_u2,log L6um AGN2_l3,log L6um AGN2_u3
0,1,5.011872,0.03162278,1.4,W0019-1046,0.14099985000000004,0.42288210149999994,0.5448948020000008,0.8577767009999997,1.0308000000000002,1.297701,0.03162278,0.04781004,0.03162278,0.06837722,0.03162278,0.06837722,46.52965748823731,0.026258942754694203,0.035478853887696005,0.05758186598529136,0.06829163775502423,0.10193941783311544,0.09618446676583403,44.58306144593525,0.1434591716827498,0.23361791610582117,0.2942855403136022,0.3653844247603075,0.5091473808257376,0.43784181580659265
1,2,4.323613,0.0,0.4451524,W0116-0505,0.8639302015000001,1.4717886435,1.3147961890000004,3.8736991530000022,1.8117269999999999,5.854741084999928,0.0,0.0,0.0,0.01584893,0.0,0.03162278,47.27990372359331,0.07609204651090096,0.11123547737166462,0.13921446247294966,0.2574851146682562,0.19395462030689004,0.3516717786318324,45.21937537758478,0.05414980054766971,0.022573276613513826,0.17645960276712458,0.07720542984682055,0.36282838268861894,0.1847906925653362
2,3,10.0,0.1,0.0,W0204-0506,2.056718,0.2073381350000023,3.6904269999999997,2.58925,4.27541458100001,5.848929999999999,0.08415107000000001,0.0,0.1,0.0,0.1,0.0,46.88544124143789,0.1060180460210276,0.0400751536274484,0.168685002138794,0.12656609134512564,0.23353148793129463,0.21897391595292248,45.001750683817434,0.4893736852521968,0.04779118863967824,0.9409168398490664,0.11401833627200375,inf,0.16757922547960646
3,4,7.570073,0.0,0.3727581,W0220+0137,1.0164122620000002,0.8421383755000003,1.680391815000001,1.9909794260000018,2.5582009999999995,2.6813896949999743,0.0,0.0,0.0,0.01,0.0,0.01995262,47.369722419587696,0.06237038884994206,0.053128685122473485,0.11583916270027572,0.11359324434213391,0.1836252294397127,0.15928259403793277,45.06967748214678,0.020948510219390926,0.012941208356167522,0.04888769619397948,0.04538183901477311,0.08577627657869868,0.10448932920981235
4,5,3.162278,0.01584893,1.4,W0831+0140,0.12021254699999995,0.48095612550000055,0.6503920000000001,0.818794,0.6503920000000001,1.3459126004999997,0.01584893,0.015773850000000002,0.01584893,0.034269789999999994,0.01584893,0.06358389,47.5443374862671,0.020763989344999345,0.04469265693582969,0.058720076402913435,0.08190459795837768,0.09746962486088506,0.12157726457132867,45.264544300139896,0.13582595260934482,0.19269284512318308,0.273721901318396,0.32796696719562846,0.39336844117614334,0.4644560132946225


In [14]:
#Final table.
wids = ['W0019$-$1046', 'W0116$-$0505', 'W0204$-$0506', 'W0220+0137', 'W0831+0140']
k = list()
for wid in wids:
    k.append(np.argwhere(d22['wid']==re.sub("\$","",wid))[0][0])

Ftab = Table()
Ftab['Short WISE ID'] = wids

ra  = [None]*len(wids)
dec = [None]*len(wids)
for i, wid in enumerate(wids):
    hmsdms1 = coords[re.sub("\$","",wid)].to_string(style='hmsdms', sep=':', precision=1)
    hmsdms2 = coords[re.sub("\$","",wid)].to_string(style='hmsdms', sep=':', precision=2)
    ra[i]  = hmsdms2.split()[0]
    dec[i] = re.sub("-","$-$",hmsdms1.split()[1])
Ftab['R.A.'] = ra
Ftab['Dec.'] = dec

Ftab['Redshift'] = d20['redshift'][k]
Ftab['Redshift'].info.format = '7.3f'

#Ftab['rmag'] = -2.5*np.log10(d21['sdssr'][k]/(3631.*u.Jy))
#Ftab['rmag'].info.format = '5.2f'

# Ftab['log L6um AGN1'] = d22['log L6um AGN1'][k]
# Ftab['log L6um AGN1'].info.format = '5.2f'

# Ftab['E(B-V) AGN1'] = d22['ebv1'][k] 
# Ftab['E(B-V) AGN1'].info.format = '5.2f'

# Ftab['log L6um AGN2'] = d22['log L6um AGN2'][k]
# Ftab['log L6um AGN2'].info.format = '5.2f'

# Ftab['E(B-V) AGN2'] = d22['ebv2'][k] 
# Ftab['E(B-V) AGN2'].info.format = '5.2f'

# We are showing things with a precision of 0.01. Errors that are formally smaller than that should be upped to 0.01 as it is most likely the gridding of the reddening parameters that is driving that. 
for cols in [('log L6um AGN1', 'log L6um AGN1'), ('E(B-V) AGN1', 'ebv1'), ('log L6um AGN2','log L6um AGN2'), ('E(B-V) AGN2','ebv2')]:
    col1, col2 = cols
    Ftab[col1] = "{:28s}".format(" ")
    d22[col2+"_u1"][d22[col2+"_u1"]<0.01] = 0.01
    d22[col2+"_l1"][d22[col2+"_l1"]<0.01] = 0.01
    for i in range(len(d22)):
        #print(col1, i, d22[col2][i], d22[col2+"_u1"][i], d22[col2+"_l1"][i])
        Ftab[col1][i] = "${:5.2f}^{{+{:4.2f}}}_{{-{:4.2f}}}$".format(d22[col2][i], d22[col2+"_u1"][i], d22[col2+"_l1"][i])

Ftab['p_ran'] = np.zeros(len(wids))
for i, wid in enumerate(wids):
    Ftab['p_ran'][i] = pran[re.sub("\$","",wid)]*1e2
Ftab['p_ran'].info.format = '5.3f'


In [15]:
Ftab.write("targets.tex", format='latex', overwrite=True)